In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('walmart-stock').getOrCreate()

In [3]:
df = spark.read.csv('/FileStore/tables/walmart_stock.csv',header=True,inferSchema=True)

In [4]:
df.show(2)

+-------------------+------------------+---------+---------+------------------+--------+------------------+
 Date| Open| High| Low| Close| Volume| Adj Close|
+-------------------+------------------+---------+---------+------------------+--------+------------------+
2012-01-03 00:00:00| 59.970001|61.060001|59.869999| 60.330002|12668800|52.619234999999996|
2012-01-04 00:00:00|60.209998999999996|60.349998|59.470001|59.709998999999996| 9593300| 52.078475|
+-------------------+------------------+---------+---------+------------------+--------+------------------+
only showing top 2 rows

In [5]:
df.columns

Out[ 5 ]: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

In [6]:
df.printSchema()

root
-- Date: timestamp (nullable = true)
-- Open: double (nullable = true)
-- High: double (nullable = true)
-- Low: double (nullable = true)
-- Close: double (nullable = true)
-- Volume: integer (nullable = true)
-- Adj Close: double (nullable = true)

In [7]:
df.describe().show()

+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
summary| Open| High| Low| Close| Volume| Adj Close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
 count| 1258| 1258| 1258| 1258| 1258| 1258|
 mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
 stddev| 6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991| 4519780.8431556|6.722609449996857|
 min|56.389998999999996| 57.060001| 56.299999| 56.419998| 2094900| 50.363689|
 max| 90.800003| 90.970001| 89.25| 90.470001| 80898100|84.91421600000001|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+

In [8]:
# to select columns by position, pass a slice of the DF created by the `.columns` attribute to the `select()` method
df.select(df.columns[0:5]).show(5)

+-------------------+------------------+---------+---------+------------------+
 Date| Open| High| Low| Close|
+-------------------+------------------+---------+---------+------------------+
2012-01-03 00:00:00| 59.970001|61.060001|59.869999| 60.330002|
2012-01-04 00:00:00|60.209998999999996|60.349998|59.470001|59.709998999999996|
2012-01-05 00:00:00| 59.349998|59.619999|58.369999| 59.419998|
2012-01-06 00:00:00| 59.419998|59.450001|58.869999| 59.0|
2012-01-09 00:00:00| 59.029999|59.549999|58.919998| 59.18|
+-------------------+------------------+---------+---------+------------------+
only showing top 5 rows

In [9]:
from pyspark.sql.functions import format_number

In [10]:
described = df.describe()

In [11]:
described.show()

+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
summary| Open| High| Low| Close| Volume| Adj Close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
 count| 1258| 1258| 1258| 1258| 1258| 1258|
 mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
 stddev| 6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991| 4519780.8431556|6.722609449996857|
 min|56.389998999999996| 57.060001| 56.299999| 56.419998| 2094900| 50.363689|
 max| 90.800003| 90.970001| 89.25| 90.470001| 80898100|84.91421600000001|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+

In [12]:
# formatting the data types of the described DF columns (which are strings)
from pyspark.sql.functions import format_number
described.select(described['summary'],
                format_number(described['Open'].cast('float'),2).alias('Open'),
                format_number(described['Low'].cast('float'),2).alias('Low')).show()

+-------+--------+--------+
summary| Open| Low|
+-------+--------+--------+
 count|1,258.00|1,258.00|
 mean| 72.36| 71.92|
 stddev| 6.77| 6.74|
 min| 56.39| 56.30|
 max| 90.80| 89.25|
+-------+--------+--------+

In [13]:
df2 = df.withColumn('HV Ratio',df['High']/df['Volume'])
df2_slice = df2.columns[7:]
df2.select(df2_slice).show()


+--------------------+
 HV Ratio|
+--------------------+
4.819714653321546E-6|
6.290848613094555E-6|
4.669412994783916E-6|
7.367338463826307E-6|
8.915604778943901E-6|
8.644477436914568E-6|
9.351828421515645E-6|
 8.29141562102703E-6|
7.712212102001476E-6|
7.071764823529412E-6|
1.015495466386981E-5|
6.576354146362592...|
 5.90145296180676E-6|
8.547679455011844E-6|
8.420709512685392E-6|
1.041448341728929...|
8.316075414862431E-6|
9.721183814992126E-6|
8.029436027707578E-6|
6.307432259386365E-6|
+--------------------+
only showing top 20 rows

In [14]:
df.orderBy(df['High'].desc()).head(1)[0][0]

Out[ 92 ]: datetime.datetime(2015, 1, 13, 0, 0)

In [15]:
from pyspark.sql.functions import min,max
df.select(max('Volume').alias('max_volume'),min('Volume').alias('min_volume')).show()

+----------+----------+
max_volume|min_volume|
+----------+----------+
 80898100| 2094900|
+----------+----------+

In [16]:
df.filter('Close < 60').count()

Out[ 98 ]: 81

In [17]:
df.filter(df['Close'] < 60).count()

Out[ 99 ]: 81

In [18]:
from pyspark.sql.functions import count
df3 = df.filter(df['Close'] < 60)

In [19]:
df3.select(count('Close').alias('Close prices over 60')).show()

+--------------------+
Close prices over 60|
+--------------------+
 81|
+--------------------+

In [20]:
# percentage of Highes over 80
(df.filter(df['High']>80).count() / df.count())*100

Out[ 108 ]: 9.141494435612083

In [21]:
from pyspark.sql.functions import corr
df.select(corr('High','Volume')).show()

+-------------------+
 corr(High, Volume)|
+-------------------+
-0.3384326061737161|
+-------------------+

In [22]:
from pyspark.sql.functions import year
yeardf = df.withColumn("year", year(df['Date']))

In [23]:
#max_df = yeardf.groupBy("Year").max()
yeardf.groupBy("year").max('High').show()

+----+---------+
year|max(High)|
+----+---------+
2015|90.970001|
2013|81.370003|
2014|88.089996|
2012|77.599998|
2016|75.190002|
+----+---------+

In [24]:
max_df.select('Year','max(High)').show()

+----+---------+
Year|max(High)|
+----+---------+
2015|90.970001|
2013|81.370003|
2014|88.089996|
2012|77.599998|
2016|75.190002|
+----+---------+